In [1]:
# Imports necessários
import re
import string
import unicodedata
import numpy as np
import pandas as pd
from tabulate import tabulate
from joblib import dump
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split


# Baixa recursos necessários do NLTK
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\augus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\augus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\augus\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# Carrega o conjunto de dados
dataset = pd.read_csv("C:\\fatec\\B2W-Reviews01\\B2W-Reviews01.csv")

# Mostra o tamanho do dataset original
print("Tamanho do dataset original:", dataset.shape[0], "\n")

# Remove linhas com valores em branco
dataset = dataset.dropna(subset=['review_text', 'overall_rating'])

# Mostra o tamanho do dataset alterado
print("Tamanho do dataset alterado:", dataset.shape[0], "\n")


Tamanho do dataset original: 132373 

Tamanho do dataset alterado: 129098 



C:\Users\augus\AppData\Local\Temp\ipykernel_7048\3311458984.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv("C:\\fatec\\B2W-Reviews01\\B2W-Reviews01.csv")


In [3]:
# Cria uma nova coluna para classificar entre comentários positivos, negativos ou neutros com base na nota:
dataset['feeling'] = np.where(dataset['overall_rating'] < 3, 'Negative', np.where(dataset['overall_rating'] == 3, 'Neutral', 'Positive'))

# Mostra os primeiros registros do dataset
print("Primeiros registros do dataset:")
print(tabulate(dataset.head(20), headers='keys', tablefmt='pipe'))


Primeiros registros do dataset:
|    | submission_date     | reviewer_id                                                      |   product_id | product_name                                                                                                                   | product_brand   | site_category_lv1       | site_category_lv2             | review_title                     |   overall_rating | recommend_to_a_friend   | review_text                                                                                                                                                                                                                                                         |   reviewer_birth_year | reviewer_gender   | reviewer_state   | feeling   |
|---:|:--------------------|:-----------------------------------------------------------------|-------------:|:----------------------------------------------------------------------------------------------------------------------------

In [4]:
# Pré-processamento dos dados
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('portuguese'))

preprocessed_texts = []  # Lista para armazenar os textos pré-processados

# Itera sobre cada texto no dataset para pré-processamento
for text in dataset['review_text']:
    if isinstance(text, str):  # Verifica se o texto é uma string
        # Converter para minúsculas
        text = text.lower()
        # Remover acentos
        text = ''.join(char for char in unicodedata.normalize('NFKD', text) if unicodedata.category(char) != 'Mn')
        # Remover números usando expressão regular
        text = re.sub(r'\d+', '', text)
        # Remover caracteres especiais (incluindo emojis)
        text = re.sub(r'[^\w\s]', '', text)
        # Remover pontuação
        text = text.translate(str.maketrans('', '', string.punctuation))
        # Remover espaços extras
        text = re.sub(r'\s+', ' ', text).strip()
        # Tokenização
        tokens = word_tokenize(text)
        # Lematização e remoção de stopwords
        tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalpha() and word.lower() not in stop_words]
        # Juntar tokens em texto novamente
        preprocessed_text = ' '.join(tokens)
        preprocessed_texts.append(preprocessed_text)  # Adicionar texto pré-processado à lista
    else:
        preprocessed_texts.append("")

# Atualiza o DataFrame com os textos pré-processados
dataset['review_text'] = preprocessed_texts

# Mostra o dataset após o pré-processamento
print("\nDataset após pré-processamento:")
print(tabulate(dataset.head(20), headers='keys', tablefmt='pipe'))



Dataset após pré-processamento:
|    | submission_date     | reviewer_id                                                      |   product_id | product_name                                                                                                                   | product_brand   | site_category_lv1       | site_category_lv2             | review_title                     |   overall_rating | recommend_to_a_friend   | review_text                                                                                                                                                                                        |   reviewer_birth_year | reviewer_gender   | reviewer_state   | feeling   |
|---:|:--------------------|:-----------------------------------------------------------------|-------------:|:-------------------------------------------------------------------------------------------------------------------------------|:----------------|:------------------------|:---------------

In [5]:
# Separar os dados em features (X) e target (Y)
X = dataset['review_text'].values
Y = dataset['feeling'].values

# Criar um objeto CountVectorizer com N-grams
ngram_vectorizer = CountVectorizer(ngram_range=(1, 3))  # Considera unigramas, bigramas e trigramas

# Vetorizar os dados de texto com N-grams
X_ngrams = ngram_vectorizer.fit_transform(X)

# Dividir os dados em conjunto de treinamento e teste
X_train, X_test, Y_train, Y_test = train_test_split(X_ngrams, Y, test_size=0.2, random_state=42)

# Criar e treinar o modelo Naive Bayes com os dados vetorizados com N-grams
model_ngrams = MultinomialNB()
model_ngrams.fit(X_train, Y_train)

# Avaliar o modelo com N-grams
accuracy_ngrams = model_ngrams.score(X_test, Y_test)
print("Acurácia:", accuracy_ngrams)


Acurácia: 0.8237800154918667


In [6]:
# Prever as classes para os dados de teste
Y_pred = model_ngrams.predict(X_test)

# Criar um DataFrame com as previsões e as verdadeiras classes de "feeling"
df_results = pd.DataFrame({'Feeling_Predicted': Y_pred, 'Feeling_True': Y_test})

# Mostrar os resultados
print("\nResultados de 'feeling' após o treinamento do modelo:")
print(df_results.head(20))  # Exibir os 20 primeiros registros



Resultados de 'feeling' após o treinamento do modelo:
   Feeling_Predicted Feeling_True
0           Positive     Positive
1           Positive     Positive
2           Positive      Neutral
3           Negative     Negative
4           Negative     Positive
5           Positive     Positive
6           Positive     Negative
7           Positive      Neutral
8           Positive     Positive
9           Positive     Positive
10          Positive     Positive
11          Positive     Positive
12          Positive     Positive
13          Negative     Negative
14          Positive     Positive
15          Positive     Positive
16          Positive     Positive
17          Positive     Positive
18          Positive      Neutral
19          Positive     Positive


In [7]:
# Salvar o modelo e o vetorizador em um único arquivo, no mesmo diretório deste notebook
dump((model_ngrams, ngram_vectorizer), 'modelo_naive_bayes.joblib')


['modelo_naive_bayes.joblib']